# Development Scratch

This notebook contains ideas/scratch/notes for development

In [106]:
import os
import sys
import django
import datetime
import json
import pandas as pd
import requests

from collections import defaultdict
from distutils.util import strtobool


os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ['DJANGO_SETTINGS_MODULE'] = 'schedule_bot.settings'

django.setup()

from availabilities.models import *
from availabilities.utils import check_availabilities
from availabilities.utils import get_availabilities_for_participants
from events.finite_state_machine import UserEventMachine
from events.models import *
from events.utils import get_all_event_participants
from events.utils import string_to_date_time, create_event
from unittest.mock import Mock

In [107]:
def send_webhook(user, msg, url):
    """
    """
    data = {
        'From': user,  # Example sender number
        'Body': msg,  # Example message body\
    }

    # Convert the data to JSON and make the POST request
    response = requests.post(
        url, 
        data=json.dumps(data), 
        headers={'Content-Type': 'application/json'}
    )
    
    return response

In [108]:
FEB_3_9AM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=9,
    tzinfo=datetime.timezone.utc
)

FEB_3_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_3_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_3_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_4_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=4, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_4_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=4, 
    hour=17,
    tzinfo=datetime.timezone.utc
)


FEB_5_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=5, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_5_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=5, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

In [109]:
UserEvent.objects.all().delete()
UserEventTime.objects.all().delete()
EventTime.objects.all().delete()
SuggestedDate.objects.all().delete()
DateAvailability.objects.all().delete()
Availability.objects.all().delete()

(0, {})

In [110]:
# Generates test data: Not available 2022/2/3 9-5PM
andrew_defaults = {
    "date_time_start": FEB_3_9AM,
    "date_time_end": FEB_3_5PM,
    "is_available": False
}

jessie_defaults = {
    "date_time_start": FEB_4_2PM,
    "date_time_end": FEB_4_5PM,
    "is_available": False
}

andrew, __ = User.objects.get_or_create(username="Andrew")
jessie, __ = User.objects.get_or_create(username="Jessie")
luna, __ = User.objects.get_or_create(username="Luna")

availability, __ = DateAvailability.objects.update_or_create(
    user=andrew,
    category="hockey_team",
    defaults=andrew_defaults
)

availability, __ = DateAvailability.objects.update_or_create(
    user=jessie,
    category="social_stuff",
    defaults=jessie_defaults
)


# Generates proposed times
proposed_times = [
    (FEB_3_2PM, FEB_3_5PM),
    (FEB_4_2PM, FEB_4_5PM),
    (FEB_5_2PM, FEB_5_5PM),
]

# Generates event
create_event(
    host_user=andrew,
    event_name="Hockey Party",
    invitees=[jessie, luna],
    proposed_times=proposed_times
)

event = Event.objects.get(
    name="Hockey Party",
)


# Peeks
event_time_df = pd.DataFrame(EventTime.objects.filter(event=event).values())
event_time_df

,id,event_id,date_time_start,date_time_end
0,71,5,2022-02-03 14:00:00+00:00,2022-02-03 17:00:00+00:00
1,72,5,2022-02-04 14:00:00+00:00,2022-02-04 17:00:00+00:00
2,73,5,2022-02-05 14:00:00+00:00,2022-02-05 17:00:00+00:00


In [111]:
user_event = UserEvent.objects.get(user=jessie, event=event)
user_event_machine = UserEventMachine(user_event)
user_event_machine.send_initial_text()

Jessie, would 2022-02-05 14:00:00+00:00 to 2022-02-05 17:00:00+00:00 work for you?
Jessie's new state: waiting_response


In [112]:
# URL of your local Django webhook endpoint
url = 'http://127.0.0.1:8000/messages/twilio/webhook'
send_webhook("Jessie", "No", url)
send_webhook("Jessie", "2022-2-17 4pm", url)
send_webhook("Jessie", "Yes", url)

<Response [500]>

In [31]:
pd.DataFrame(UserEvent.objects.values())

,id,state,user_id,event_id,is_required,is_host
0,52,waiting_for_others,1,5,True,True
1,53,waiting_for_others,2,5,True,False
2,54,waiting_response,3,5,True,False


In [104]:

# import parsedatetime
# import pytz

# def string_to_date_time(input_string):
#     """
#     Translates input_string into datetime
#     """
#     # Convert string to structtime
#     cal = parsedatetime.Calendar()
#     struct_time, __ = cal.parse(input_string)

#     # Convert structtime to datetime
#     date_time = datetime.datetime(*struct_time[:6])

#     return date_time


# date = string_to_date_time("2024-01-02 2pm")

# # pst_timezone = pytz.timezone('America/Los_Angeles')

datetime.datetime(2024, 1, 2, 14, 0)

In [105]:
# pst_timezone.localize(date)

datetime.datetime(2024, 1, 2, 14, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)